# Tasking Requests (TEMPLATE)
This script is a template for placing, reviewing, approving, and checking the status of tasking requests for a single-point or single-polygon AOI based on a set of defined constraints. 

The cells that require the user to input parameters prior to running are marked with `USER INPUT REQUIRED` in the header. Searching for `USER` in the notebook will also identify where the user must enter inputs.

* **Author:** Hayley Pippin
* **Last updated:** August 9, 2024
* **Required input(s):**
    * `credentials.json`: JSON containing the user's Capella Console credentials.
    * `JSON` or `GEOJSON` single-point or single-polygon file
* **Output(s):**
    * NA

## Setup
Unless otherwise noted, these cells must be run every time you use this script. 

### Install packages
The following cell **only needs to be run once** if packages are not already installed. Uncomment any of the following lines to install the necessary packages.

In [ ]:
#!pip install requests
#!pip install json
#!pip install pandas

### Import packages and define helper functions

In [ ]:
import requests
import json
import csv
import os
import datetime
import pandas as pd

# Helper function to print formatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

### Authentication (USER INPUT REQUIRED)
This cell needs to be run hourly to re-authenticate with the Capella system.

In [ ]:
# Load username and password
with open('WRITE PATH HERE') as f: # USER: Input path to credentials.json file.
    data = json.load(f)
    username = data['username']
    password = data['password']

# Get a valid token from the auth service
r = requests.post("https://api.capellaspace.com/token", 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
access_token = r.json()["accessToken"]
# p(accessToken)

# GET user ID and org ID
headers = {'Authorization':'Bearer ' + access_token}
r = requests.get("https://api.capellaspace.com/user", headers=headers)
user_id = r.json()["id"]
org_id = r.json()["organizationId"]
#p(r.json())

# Print user and org ID
print('User email: ', r.json()['email'], '\nOrganization: ', r.json()['organization']['name'], '\nEnvironment: ', r.json()['apiEnvironmentRole'])

## Submit Tasking Request

### Define tasking request input parameters (USER INPUT REQUIRED)

In [ ]:
# Set AOI (must be SINGLE POINT or SINGLE POLYGON in JSON or GEOJSON format)
aoi_path = "WRITE PATH HERE" # USER: Input path to AOI.
aoi = json.load(open(aoi_path))
aoi_geom = aoi['features'][0]['geometry'] # Get the geometry of the AOI

# Name and description
request_name = '' # USER: Set the name of the request.
request_description = '' # USER: Set a description for the request.

# Tasking request window
# Default: 14-day window starting at current UTC date/time
start_date = datetime.datetime.utcnow()
end_date = start_date + datetime.timedelta(days = 14)
# USER: To change the window from the default, uncomment and set the start_date and end_date for your desired window in UTC in YYYY,M,D format.
#       Tasking request window must be at least 1 day in length, and may not be fewer days in length than the tier specified below.
# start_date = datetime.date(YYYY,M,D)
# end_date = datetime.date(YYYY,M,D)
window_open = start_date.strftime("%Y-%m-%dT%H:%M:%SZ") # Format start date
window_close = end_date.strftime("%Y-%m-%dT%H:%M:%SZ") # Format end date

# Collection tier
# USER: Set the prioritization of your task (NOTE: NOT the same as the tasking request window length.)
tier = "flexible" # options: urgent, priority, standard, flexible

# Imaging mode and footprint size
# USER: Set the desired imaging mode and image footprint. Ensure the img_length and img_width fit the guidelines listed below.
# Footprint size guidelines:
# - Spotlight: length = 5000, width = 5000
# - Sliding Spotlight: length = 10000, width = 5000
# - Stripmap: length = 20000, width = 5000-10000
img_mode = "stripmap" # Options: spotlight, sliding_spotlight, stripmap
img_length = 20000 # in meters
img_width = 5000 # in meters

# Product category and look angle range
# USER: Select category and look angle range. Ensure the look angle min and max fit the guidelines listed below.
# Product category/look angle guidelines:
# - Standard: min = 25, max = 50
# - Extended: min = 15, max = 50
# - Custom: min >= 5, max <= 50
product_cat = "extended" # Options: standard, extended, custom
look_angle_min = 15 # Full: 5, Extended: 15, Standard: 25
look_angle_max = 50 # Max: 50

# Configuration constraints
# Default: Any configuration.
# USER: Choose from the available options to limit the possible configuration(s) your image could be collected as, or leave as default.
orbital_plane = [45, 53, 97] # Options: 45, 53, 97
orbit_state = "either" # Options: ascending, descending, either
look_direction = "either" # Options: left, right, either

# Pre-approval
# USER: Set to "false" if you NEED TO REVIEW TASK PRICING before submitting the task.
pre_approval = "false" # Options: true or false

### Create and `POST` tasking request

In [ ]:
# Create tasking request
tasking_request = {
  "type": "Feature",
  "geometry": aoi_geom,
  "properties": {
    "taskingrequestDescription": request_description,
    "taskingrequestName": request_name,
    "windowOpen": window_open,
    "windowClose": window_close,
    "collectionTier": tier,
    "collectConstraints": {
      "lookDirection": look_direction,
      "ascDsc": orbit_state,
      "orbitalPlanes": orbital_plane,
      #"localTime": [
      #  [
      #    0,
      #    86400
      #  ]
      #],
      "offNadirMin": look_angle_min,
      "offNadirMax": look_angle_max,
      "collectMode": img_mode,
      "imageLength": img_length,
      "imageWidth": img_width,
      # "grrMin": 0.5,
      # "grrMax": 0.7,
      # "azrMin": 0.5,
      # "azrMax": 0.5,
      # "numLooks": num_looks,
      # "polarization": "HH"
    },
    "productCategory": product_cat,
    "preApproval": pre_approval
  }
}

# POST tasking request
r = requests.post("https://api.capellaspace.com/task/", json = tasking_request, headers = headers)
#p(r.json())

# Define the tasking request ID and name for later use
tr_id = r.json()["properties"]["taskingrequestId"]
tr_name = r.json()["properties"]["taskingrequestName"]

# Print the ID and tasking request response to ensure task was submitted successfully
print("Response: ", r.json(), "\nRequest ID: ", tr_id)

## Review and Approve SINGLE Tasking Requests
If `pre_approval` was set to `false` when submitting the tasking request, the task pricing and parameters need to be reviewed and approved before the task is submitted to the scheduler. Code can easily be adapted for repeat requests. **This step is not required if `pre_approval` was set to `true`.**

### Review the request (USER INPUT OPTIONAL)

In [ ]:
# Uncomment the tr_id line below to define a specific tasking ID. 
# If you just submitted a task and want to review the same task, leave the line commented out.
# tr_id = "WRITE TASK ID HERE" # USER: Optionally define specific tasking ID here.

r = requests.get("https://api.capellaspace.com/task/" + tr_id, headers=headers) # Task search
#p(r.json()) # Uncomment to view full response

tr_name = r.json()["properties"]["taskingrequestName"]
tr_status = r.json()["properties"]["statusHistory"] 
tr_cost_est = r.json()["properties"]["order"]["summary"]

print("Request ID: ", tr_id, "\nTask name: ", tr_name, "\nStatus history: ")
p(tr_status[0]) # To view only the most recent status, use p(tr_status[0])
print("Cost per Image Estimate: ", tr_cost_est)

### Approve or reject the request (USER INPUT REQUIRED)

In [ ]:
# USER: Choose to approve or reject the task.
params = {'status': 'approved'} # Options: approved, rejected

r = requests.patch("https://api.capellaspace.com/task/" + tr_id + "/status", headers=headers, json=params)
p(r.json())

## Check Tasking Request Status (USER INPUT OPTIONAL)
Check the status of your tasking request. You can view the entire history of the task or the most recent update. For more information on tasking and collect status, read through [this support page](https://support.capellaspace.com/hc/en-us/articles/360059270751-What-are-the-tasking-and-collect-statuses-).

In [ ]:
# Uncomment the tr_id line below to define a specific tasking ID. 
# If you just submitted a task and want to check the status of the same task, leave the line commented out.
# tr_id = "WRITE TASK ID HERE" # USER: Optionally define specific tasking ID here.

r = requests.get("https://api.capellaspace.com/task/" + tr_id, headers=headers)
#p(r.json()) # Uncomment to view full response

tr_name = r.json()["properties"]["taskingrequestName"]
tr_status = r.json()["properties"]["statusHistory"]

print("Request ID: ", tr_id, "\nTask name: ", tr_name, "\nStatus history: ")
p(tr_status) # To view only the most recent status, use p(tr_status[0])